# Venue Search Agentic Pipeline

## Initial Imports

In [1]:
# Warning control
import warnings
warnings.filterwarnings('ignore')

# Load environment variables
from helper import load_env
load_env()

import os
import yaml
from crewai import Agent, Task, Crew

## Load API tokens for our 3rd party APIs

In [2]:
os.environ['OPENAI_MODEL_NAME'] = 'o1-mini'

## Loading Tasks and Agents YAML files

In [3]:
# Define file paths for YAML configurations
files = {
    # 'data_collection_agents': 'config/agents/data_collection_agents.yaml',
    # 'data_collection_tasks': 'config/tasks/data_collection_tasks.yaml',
    'location_analysis_agents': 'config/agents/location_analysis_agents.yaml',
    'location_analysis_tasks': 'config/tasks/location_analysis_tasks.yaml',
    'email_engagement_agents': 'config/agents/email_engagements_agents.yaml',
    'email_engagement_tasks': 'config/tasks/email_engagement_tasks.yaml',
    'feature_extraction_agents': 'config/agents/feature_extraction_agents.yaml',
    'feature_extraction_tasks': 'config/tasks/feature_extraction_tasks.yaml',
    'scoring_agents': 'config/agents/scoring_agents.yaml',
    'scoring_tasks': 'config/tasks/scoring_tasks.yaml',
    'reporting_agents': 'config/agents/reporting_agents.yaml',
    'reporting_tasks': 'config/tasks/reporting_tasks.yaml'
}

# Load configurations from YAML files
configs = {}
for config_type, file_path in files.items():
    with open(file_path, 'r') as file:
        configs[config_type] = yaml.safe_load(file)

# Assign loaded configurations to specific variables
# data_collection_agents_config = configs['data_collection_agents']
# data_collection_tasks_config = configs['data_collection_tasks']
location_analysis_agents_config = configs['location_analysis_agents']
location_analysis_tasks_config = configs['location_analysis_tasks']
feature_extraction_agents_config = configs['feature_extraction_agents']
feature_extraction_tasks_config = configs['feature_extraction_tasks']
scoring_agents_config = configs['scoring_agents']
scoring_tasks_config = configs['scoring_tasks']
email_engagement_agents_config = configs['email_engagement_agents']
email_engagement_tasks_config = configs['email_engagement_tasks']
reporting_agents_config = configs['reporting_agents']
reporting_tasks_config = configs['reporting_tasks']

## Create Pydantic Models for Structured Output

In [4]:
from pydantic import BaseModel, Field
from typing import Dict, Optional, List, Set, Tuple

# Venue Search Pydantic Models
class Venue(BaseModel):
    name: str = Field(..., description="The name of the venue.")
    location: str = Field(..., description="The location of the venue.")
    capacity: int = Field(..., description="The capacity of the venue in terms of attendees.")
    amenities: List[str] = Field(..., description="A list of amenities available at the venue.")
    accessibility: str = Field(..., description="The accessibility of the venue for attendees with disabilities.")
    pricing: float = Field(..., description="The pricing of the venue per attendee.")
    website: str = Field(..., description="The website of the venue.")
    contact_info: str = Field(..., description="The contact information of the venue.")
    reviews: List[str] = Field(..., description="A list of reviews of the venue.")
    photos: List[str] = Field(..., description="A list of photos of the venue.")
    capacity: int = Field(..., description="The capacity of the venue in terms of attendees.")
    amenities: List[str] = Field(..., description="A list of amenities available at the venue.")
    accessibility: str = Field(..., description="The accessibility of the venue for attendees with disabilities.")
    pricing: float = Field(..., description="The pricing of the venue per attendee.")
    website: str = Field(..., description="The website of the venue.")
    contact_info: str = Field(..., description="The contact information of the venue.")
    reviews: List[str] = Field(..., description="A list of reviews of the venue.")
    photos: List[str] = Field(..., description="A list of photos of the venue.")

class VenueScore(BaseModel):
    score: int = Field(..., ge=0, le=100, description="The final score assigned to the venue (0-100).")
    scoring_criteria: List[str] = Field(..., description="The criteria used to determine the venue's score.")
    validation_notes: Optional[str] = Field(None, description="Any notes regarding the validation of the venue score.")

class VenueScoringResult(BaseModel):
    venue: Venue = Field(..., description="The venue object with all its attributes.")
    venue_score: VenueScore = Field(..., description="The calculated score and related information for the venue.") 



## Importing Tools

In [5]:
from crewai_tools import SerperDevTool, ScrapeWebsiteTool

## Lead Qualification Crew, Agents and Tasks

In [6]:
# Creating Agents

# data_collection_agent = Agent(
#   config=data_collection_agents_config['data_collector'],
#   tools=[SerperDevTool(), ScrapeWebsiteTool()]
# )

location_analysis_agent = Agent(
  config=location_analysis_agents_config['location_analyst'],
  tools=[SerperDevTool(), ScrapeWebsiteTool()]
)

feature_extraction_agent = Agent(
  config=feature_extraction_agents_config['feature_extractor'],
  tools=[SerperDevTool(), ScrapeWebsiteTool()]
)

scoring_agent = Agent(
  config=scoring_agents_config['scoring_agent'],
  tools=[SerperDevTool(), ScrapeWebsiteTool()]
)

email_engagement_agent = Agent(
  config=email_engagement_agents_config['email_agent'],
  tools=[SerperDevTool(), ScrapeWebsiteTool()]
)

reporting_agent = Agent(
  config=reporting_agents_config['reporting_agent'],
  tools=[SerperDevTool(), ScrapeWebsiteTool()]
)

# Creating Tasks
# data_collection_task = Task(
#   config=data_collection_tasks_config['collect_conference_data'],
#   agent=data_collection_agent
# )

location_analysis_task = Task(
  config=location_analysis_tasks_config['identify_venues'],
  agent=location_analysis_agent
)

feature_extraction_task = Task(
  config=feature_extraction_tasks_config['extract_features'],
  agent=feature_extraction_agent
)

scoring_task = Task(
  config=scoring_tasks_config['score_venues'],
  agent=scoring_agent,
  context=[location_analysis_task, feature_extraction_task],
  output_pydantic=VenueScoringResult
)

email_engagement_task = Task(
  config=email_engagement_tasks_config['generate_emails'],
  agent=email_engagement_agent,
  context=[scoring_task],
  output_pydantic=VenueScoringResult
)

reporting_task = Task(
  config=reporting_tasks_config['generate_report'],
  agent=reporting_agent,
  context=[scoring_task],
  output_pydantic=VenueScoringResult
)

# Creating Crew
venue_scoring_crew = Crew(
  agents=[
    # data_collection_agent,
    location_analysis_agent,
    feature_extraction_agent,
    scoring_agent,
    email_engagement_agent,
    reporting_agent
  ],
  tasks=[
    # data_collection_task,
    location_analysis_task,
    feature_extraction_task,
    scoring_task,
    email_engagement_task,
    reporting_task
  ],
  verbose=True
)

## Creating Complete Venue Search Flow

In [7]:
from crewai import Flow
from crewai.flow.flow import listen, start

class VenueSearchPipeline(Flow):
    @start()
    def collect_location_data(self):
        # Collect initial address
        locations = [
            {
                "address": "333 Adams St, Brooklyn, NY 11201, United States",
                "radius_km": 10,
                "conference_dates": "2025-01-01 to 2025-01-02", 
                "expected_attendance": 100
            }
        ]
        return locations

    @listen(collect_location_data) 
    def analyze_locations(self, locations):
        # Add venue_name and specific_features to each location before passing to crew
        locations_with_venue = [
            {
                **location, 
                "venue_name": "TBD",
                "specific_features": []  # Add empty specific_features list
            }
            for location in locations
        ]
        analysis = venue_scoring_crew.kickoff_for_each(locations_with_venue)
        self.state["location_analysis_results"] = analysis
        return analysis

    @listen(analyze_locations)
    def extract_features(self, analysis):
        # Extract and analyze venue features
        return venue_scoring_crew.kickoff()

    @listen(extract_features)
    def score_venues(self, features):
        scores = venue_scoring_crew.kickoff()
        return [venue for venue in scores if venue['venue_score'].score > 80]

    @listen(score_venues)
    def generate_emails(self, scored_venues):
        venue_contacts = [
            {
                "venue_name": venue.get("name", ""),
                **venue.to_dict()
            } 
            for venue in scored_venues
        ]
        emails = venue_scoring_crew.kickoff_for_each(venue_contacts)
        return emails

    @listen(generate_emails)
    def generate_report(self, outreach_data):
        # Generate final report with venue analysis and recommendations
        return venue_scoring_crew.kickoff()

flow = VenueSearchPipeline()

## Plotting the Flow

In [8]:
VenueSearchPipeline().plot()

Plot saved as crewai_flow.html


In [9]:
from IPython.display import IFrame

IFrame(src='./crewai_flow.html', width='150%', height=600)

## Flow Kickoff

In [10]:
import asyncio

# Create a new event loop and run once
# loop = asyncio.new_event_loop()
# asyncio.set_event_loop(loop)
# report = loop.run_until_complete(flow.kickoff_async())
# loop.close()
report = await flow.kickoff_async()


2024-12-22 19:40:23,116 - 8178152000 - __init__.py-__init__:537 - WARNING: Overriding of current TracerProvider is not allowed


# Agent: Location Analysis Agent
## Task: Using the provided 333 Adams St, Brooklyn, NY 11201, United States, identify all potential venues within  10km radius that could host technology-related events. Consider only venues  that are appropriate for professional gatherings.



# Agent: Location Analysis Agent
## Thought: Thought: I need to find venues within a 10km radius of 333 Adams St, Brooklyn, NY 11201 that are suitable for professional technology-related events.
## Using tool: Search the internet
## Tool Input: 
"{\"search_query\": \"professional venues for technology events within 10 km of 333 Adams St, Brooklyn, NY 11201\"}"
## Tool Output: 

Search results: Title: THE BEST 10 Venues & Event Spaces near METROTECH CENTER ...
Link: https://www.yelp.com/search?cflt=venues&find_loc=MetroTech+Center%2C+Brooklyn%2C+NY+11201
Snippet: Best Venues & Event Spaces in Brooklyn, NY - Vine, Hide Rooftop, Ace Hotel - Brooklyn, Studio 220, Skylight One Hanson, Wild East Brewing Co., ...
---
Ti

2024-12-22 19:49:34,583 - 8178152000 - llm.py-llm:170 - ERROR: LiteLLM call failed: litellm.RateLimitError: RateLimitError: OpenAIException - Error code: 429 - {'error': {'message': 'You exceeded your current quota, please check your plan and billing details. For more information on this error, read the docs: https://platform.openai.com/docs/guides/error-codes/api-errors.', 'type': 'insufficient_quota', 'param': None, 'code': 'insufficient_quota'}}




LiteLLM.Info: If you need to debug this error, use `litellm.set_verbose=True'.

# Agent: Location Analysis Agent
## Task: Using the provided {address}, identify all potential venues within  {radius_km}km radius that could host technology-related events. Consider only venues  that are appropriate for professional gatherings.



2024-12-22 19:49:35,942 - 8178152000 - llm.py-llm:170 - ERROR: LiteLLM call failed: litellm.RateLimitError: RateLimitError: OpenAIException - Error code: 429 - {'error': {'message': 'You exceeded your current quota, please check your plan and billing details. For more information on this error, read the docs: https://platform.openai.com/docs/guides/error-codes/api-errors.', 'type': 'insufficient_quota', 'param': None, 'code': 'insufficient_quota'}}




LiteLLM.Info: If you need to debug this error, use `litellm.set_verbose=True'.

# Agent: Location Analysis Agent
## Task: Using the provided {address}, identify all potential venues within  {radius_km}km radius that could host technology-related events. Consider only venues  that are appropriate for professional gatherings.



2024-12-22 19:49:37,718 - 8178152000 - llm.py-llm:170 - ERROR: LiteLLM call failed: litellm.RateLimitError: RateLimitError: OpenAIException - Error code: 429 - {'error': {'message': 'You exceeded your current quota, please check your plan and billing details. For more information on this error, read the docs: https://platform.openai.com/docs/guides/error-codes/api-errors.', 'type': 'insufficient_quota', 'param': None, 'code': 'insufficient_quota'}}




LiteLLM.Info: If you need to debug this error, use `litellm.set_verbose=True'.

[Flow._execute_single_listener] Error in method extract_features: litellm.RateLimitError: RateLimitError: OpenAIException - Error code: 429 - {'error': {'message': 'You exceeded your current quota, please check your plan and billing details. For more information on this error, read the docs: https://platform.openai.com/docs/guides/error-codes/api-errors.', 'type': 'insufficient_quota', 'param': None, 'code': 'insufficient_quota'}}


Traceback (most recent call last):
  File "/opt/homebrew/Caskroom/miniforge/base/envs/agents/lib/python3.11/site-packages/litellm/llms/OpenAI/openai.py", line 860, in completion
    raise e
  File "/opt/homebrew/Caskroom/miniforge/base/envs/agents/lib/python3.11/site-packages/litellm/llms/OpenAI/openai.py", line 796, in completion
    self.make_sync_openai_chat_completion_request(
  File "/opt/homebrew/Caskroom/miniforge/base/envs/agents/lib/python3.11/site-packages/litellm/llms/OpenAI/openai.py", line 657, in make_sync_openai_chat_completion_request
    raise e
  File "/opt/homebrew/Caskroom/miniforge/base/envs/agents/lib/python3.11/site-packages/litellm/llms/OpenAI/openai.py", line 639, in make_sync_openai_chat_completion_request
    raw_response = openai_client.chat.completions.with_raw_response.create(
                   ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/opt/homebrew/Caskroom/miniforge/base/envs/agents/lib/python3.11/site-packages/openai/_legacy_resp

## Usage Metrics and Costs

Let’s see how much it would cost each time if this crew runs at scale.

In [17]:
import pandas as pd

# Convert UsageMetrics instance to a DataFrame
df_usage_metrics = pd.DataFrame([report[0].token_usage.dict()])

# Calculate total costs
costs = 0.150 * df_usage_metrics['total_tokens'].sum() / 1_000_000
print(f"Total costs: ${costs:.4f}")

# Display the DataFrame
df_usage_metrics

Total costs: $0.0219


,total_tokens,prompt_tokens,cached_prompt_tokens,completion_tokens,successful_requests
0,145673,56702,27264,88971,15


## Inspecting Results

In [44]:
import json
report[0]
# Accessing the crew output
venue_scoring_result = report[0].raw
print(f"Raw Output: {venue_scoring_result}")
# if report[0].json_dict:
#     print(f"JSON Output: {json.dumps(report[0].json_dict, indent=2)}")
# if report[0].pydantic:
    # print(f"Pydantic Output: {report[0].pydantic}")
print(f"Tasks Output: {report[0].tasks_output}")
print(f"Token Usage: {report[0].token_usage}")

Raw Output: [
  {
    "venue": {
      "name": "New York Marriott at the Brooklyn Bridge",
      "location": "333 Adams St, Brooklyn, NY 11201",
      "capacity": 500,
      "amenities": [
        "Multiple event spaces",
        "State-of-the-art audiovisual equipment",
        "High-speed Wi-Fi",
        "On-site parking",
        "Valet parking",
        "In-house catering",
        "Accessible entrances and restrooms",
        "Elevator access",
        "Assistive listening devices"
      ],
      "accessibility": "Fully accessible entrances and restrooms compliant with ADA standards. Elevator access to all floors and event spaces. Assistive listening devices and accessible seating arrangements available upon request.",
      "pricing": 0.0,
      "website": "https://www.marriott.com/en-us/hotels/nycbk-new-york-marriott-at-the-brooklyn-bridge/overview/",
      "contact_info": "+1 718-246-7000",
      "reviews": [],
      "photos": []
    },
    "venue_score": {
      "score": 95,
 

In [48]:
import pandas as pd
from IPython.display import display, HTML

venue_scoring_result = report[0].raw
print(type(venue_scoring_result))

# Create a dictionary with the nested structure flattened
data = {
    'Name': venue_scoring_result,
    'Location': venue_scoring_result,
    'Capacity': venue_scoring_result,
    'Amenities': venue_scoring_result,
    'Accessibility': venue_scoring_result,
    'Pricing': venue_scoring_result,
    'Website': venue_scoring_result,
    'Contact Info': venue_scoring_result,
    'Reviews': venue_scoring_result,
    'Photos': venue_scoring_result,
    'Venue Score': venue_scoring_result,
    'Scoring Criteria': venue_scoring_result,
    'Validation Notes': venue_scoring_result
}

# Convert the dictionary to a DataFrame
df = pd.DataFrame.from_dict(data, orient='index', columns=['Value'])

# Reset the index to turn the original column names into a regular column
df = df.reset_index()

# Rename the index column to 'Attribute'
df = df.rename(columns={'index': 'Attribute'})

# Create HTML table with bold attributes and left-aligned values
html_table = df.style.set_properties(**{'text-align': 'left'}) \
                     .format({'Attribute': lambda x: f'<b>{x}</b>'}) \
                     .hide(axis='index') \
                     .to_html()

# Display the styled HTML table
display(HTML(html_table))

<class 'str'>


## Results

In [ ]:
import textwrap

result_text = report[0].raw
wrapped_text = textwrap.fill(result_text, width=80)
print(wrapped_text)